In [0]:
import pickle as pkl
import numpy as np
import keras
import boto3
import numpy as np
import pandas as pd
import os
import json
from datetime import timedelta
from dateutil import parser
import boto3
import urllib.request
import datetime

In [0]:
####### reading in files


dbfile = open('/content/drive/My Drive/Fresh_air_data/x_master_new.pkl', 'rb')      
X = pkl.load(dbfile)
dbfile.close() 

dbfile = open('/content/drive/My Drive/Fresh_air_data/y_master_new.pkl', 'rb')      
Y = pkl.load(dbfile)
dbfile.close() 

X.shape

## Review Label to Words 
    * 1:'Smoking induced',
    * 2:'Smoking induced',
    * 3:'Smoking induced',
    * 10:'Inconclusive',
    * 11:'Inconclusive',
    * 13:'Inconclusive',
    * 20:'Environmental Anomalies',
    * 30:'Environmental Anomalies',
    * 31:'Environmental Anomalies',
    * 32:'Environmental Anomalies',
    * 33:'Environmental Anomalies',
    * 34:'Environmental Anomalies',
    * 35:'Environmental Anomalies',
    * 36:'Environmental Anomalies',
    * 45:'Environmental Anomalies',
    * 50:'Environmental Anomalies', 



In [0]:

####### just smoking and Evironment  Variables ########

x_bin=[]
y_bin=[]
smoking_event=0
non_smoking_event=0


for i, y in enumerate(Y):
    if y in [1,2,3,31,32,33,34,35,36,45,50]:
      if X[i].shape == (760, 4):
        if y in [1,2,3]:
          y_bin.append(1)
          x_bin.append(X[i])
          smoking_event+=1
        elif y in [31,32,33,34,35,36,45,50]:
          y_bin.append(0)
          x_bin.append(X[i])
          non_smoking_event+=1
        
    # if i==500 :break

x_bin=np.array(x_bin)
y_bin=np.array(y_bin)

In [0]:
print(non_smoking_event)
print(smoking_event)

In [0]:
import datetime
now=datetime.datetime.now

In [0]:
now()

In [0]:
%load_ext tensorboard
# from keras.layers import SpatialDropout2D, Conv2D,ConvLSTM2D,CuDNNLSTM,Flatten


In [0]:
####### reading in file #########

#### part two Creating 
from keras.models import Sequential
from keras.layers import Dense  
from keras.layers import LSTM
from keras.layers import SpatialDropout2D, Conv2D,ConvLSTM2D, CuDNNLSTM
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
import tensorflow as tf

BatchSize=32
Epochs=1000
responce_y=y_bin
responce_x=x_bin

################# call backs #########################
checkpoint=ModelCheckpoint( "/content/drive/My Drive/Fresh_air_data/models/model-{epoch:02d}-{val_binary_accuracy:.4f}.hdf5"
, monitor='val_accuracy', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1
)

tesnor_board=TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=BatchSize, write_graph=True, 
                         write_grads=True, embeddings_freq=0, embeddings_layer_names=None, 
                        embeddings_metadata=None, embeddings_data=None, update_freq='batch')

early_stop=EarlyStopping(
monitor='val_loss', min_delta=.0001, patience=2, verbose=0, mode='auto', baseline=None, restore_best_weights=False
)



################# Building the Model ########################
# Initializing the RNN
model = Sequential()

# first Lstm Layer
model.add(CuDNNLSTM(units = 50, return_sequences=True, input_shape=(responce_x.shape[1], responce_x.shape[2])))
model.add(Dropout(.1)) #### .2 is the hueristic

# # second Lstm Layer
model.add(CuDNNLSTM(units = 25, return_sequences=False))
model.add(Dropout(.1)) #### .2 is the hueristic

# # #third Lstm Layer
# model.add(LSTM(units = 50, return_sequences=True))
# # model.add(Dropout(.1)) #### .2 is the hueristic

# fouth Lstm Layer
# model.add(LSTM(units = 50, return_sequences=False))
# model.add(Dropout(.1)) #### .2 is the hueristic

model.add(Dense(units=10, activation='relu'))
### adding in the output layer
model.add(Dense(units=1, activation='sigmoid'))

### compile
model.compile(optimizer='adam', loss = 'binary_crossentropy',metrics=['accuracy','binary_accuracy'])  #### RMSprop or Adam is recomended for RNN

# fiting the RNN to the Trainging Set
hist=model.fit(responce_x, responce_y, epochs=Epochs, batch_size=BatchSize,
            validation_split=.1, use_multiprocessing=True, callbacks=[checkpoint,tesnor_board])








In [0]:
x=now()
x

In [0]:
####### reading in file #########

#### part two Creating 
from keras.models import Sequential
from keras.layers import Dense  
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
import tensorflow as tf

BatchSize=32
Epochs=1000
responce_y=y_bin
responce_x=x_bin

################# call backs #########################
checkpoint=ModelCheckpoint( "/content/drive/My Drive/Fresh_air_data/models/model-{epoch:02d}-{val_binary_accuracy:.4f}.hdf5"
, monitor='val_accuracy', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1
)

tesnor_board=TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=BatchSize, write_graph=True, 
                         write_grads=True, embeddings_freq=0, embeddings_layer_names=None, 
                        embeddings_metadata=None, embeddings_data=None, update_freq='batch')

early_stop=EarlyStopping(
monitor='val_loss', min_delta=.0001, patience=2, verbose=0, mode='auto', baseline=None, restore_best_weights=False
)

import tensorflow as tf
new_model = tf.keras.models.load_model('/content/drive/My Drive/Fresh_air_data/models/model-01-0.7197.hdf5')

new_model.fit(responce_x, responce_y, epochs=Epochs, batch_size=BatchSize,
            validation_split=.1, use_multiprocessing=True, callbacks=[checkpoint,tesnor_board])

In [0]:
############ clearing out old data
# if tf.gfile.Exists('logs'):
#    tf.gfile.DeleteRecursively('logs') 

%tensorboard --logdir logs